This notebook implements a Retrieval-Augmented Generation (RAG) pipeline. RAG allows an AI to answer questions based on specific data (like a website or a PDF) rather than just its general training data

In [ ]:
# !pip install langchain_community langchainhub chromadb langchain langchain-openai

In [9]:
pip install langchain_community chromadb langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#chromadb'

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
os.environ.get('OPENAI_API_KEY')

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
splits = text_splitter.split_documents(docs)

In [ ]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Ongoing LIVE CourseHands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AIJoin Anytime!Get LifeTime AccessAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Projects using TensorFlow Metrics Visualization using TensorBoard Mini Project - Imple

In [ ]:
print(len(splits))

11


In [ ]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

In [ ]:
print(vectorstore._collection.count())

22


In [ ]:
print(vectorstore._collection.get())

{'ids': ['80c71ae3-b35e-4ae1-8cd0-261e41888cde', '34474c35-e8bb-4064-9a66-fb0edb0976a1', 'ce3963a7-8572-4a6f-8f51-7d0719457e88', 'f97cbe75-3624-494d-89cd-bbdbe5fbc390', '6e530704-062d-4004-882b-9a1dd0646f2a', 'c22d28e6-17fb-4e20-8f3d-f2e1da3faa04', '750c6aeb-ac1e-456f-b086-852911ec7269', '75ef7481-28cc-41ab-be91-b3f246e06f1e', 'e969737b-95ed-451e-98c7-33ee5ea60c54', '5663852c-7316-437e-b2cb-1217eade0763', 'd4eec53e-5124-4357-9e63-82b1c3c7998e', 'cba99ed2-9a32-4f8a-9110-7aa72543d58c', '57de8aa8-f36c-4a8e-9606-3b1f56d4b97f', '09cd5ab6-34fd-4ac5-9e64-c58abdf95393', '0514ea5e-10fc-42ab-bc09-973bc57ede44', 'be2b657a-ef2b-4d3f-9d4f-979b159c969b', 'a7b0ed96-2b99-42dd-a284-3ed991e823ea', '067e1bd8-af7c-496b-b3f8-1e338b3708a4', '2acccd21-a4b9-4337-b10b-ffd0295f6139', '9ae70cb5-0749-4551-ba95-682e84eeb637', 'cbb40332-3b88-46a8-b7df-2deb298249c1', 'd6e9c104-2a7f-4c72-abfa-e289f5aaf06a'], 'embeddings': None, 'documents': ['Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimo

In [ ]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['28651d9a-ab51-41f8-ab83-e68285623c4e'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['054dee19-19ed-4574-bc51-511060fd707a'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['2fd71cb4-835a-43c5-b920-b7e1be51c450'], include=["embeddings", "documents"]))


Collection 1 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 2 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 3 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()

In [ ]:
# from langchain_community.llms import HuggingFaceHub
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0.5, "max_length":512}, task="text2text-generation")

from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
import os
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [ ]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

Token indices sequence length is longer than the specified maximum sequence length for this model (777 > 512). Running this sequence through the model will result in indexing errors


'Can I watch the class recordings or go through notes offline?Do you provide certificate for the course?Can I get invoices for reimbursement?I have more questions, how can I reach out to you?EducosysJoin our Discord for updates, upcoming courses and exclusive offers.ConnectFeaturesCoursesBundle CoursesTestimonialsFAQLegalPrivacy PolicyTerms of Services 2025 ITTI Tech. All rights reserved.'

In [ ]:
rag_chain.invoke("Are the testimonials for the course available? Name the studenst who have shared testimonials")

'this course to anyone interested in AI and ML!Read moreManika KaushikSenior Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.'

In [ ]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes, certificates are provided for the course.'

In [ ]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes, I have more questions, how can I reach out to you?EducosysJoin our Discord for updates, upcoming courses and exclusive offers.ConnectFeaturesCoursesBundle CoursesTestimonialsFAQLegalPrivacy PolicyTerms of Services 2025 ITTI Tech. All rights reserved. this course to anyone interested in AI and ML!Read moreManika KaushikSenior Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.Frequently asked questionsIs this a Live or Recorded Course?When will the next Live batch be launched?What if I am interested in learning Live only?What are the prerequisites for the course?Is Machine Learning pre-requisite for the course?How many projects will we work on? Can I add these to resume?Is this course for freshers or for experienced people?How will you be able to cover so much in 7 weeks?What languages do we use to build the projects?Can I watch the classes only on the phone without a Lapt

In [ ]:
rag_chain.invoke("What all projects are covered in the course?")

'The projects covered in the course include implementing GANs to generate handwritten digits, training a VAE to generate faces using the CelebA dataset, building a transformer from scratch, and performing sentiment analysis using BERT. The course also covers topics such as attention mechanisms, transformers, fine-tuning techniques, and vector databases. It offers hands-on learning experiences and allows participants to work on mini and major projects to enhance their skills in generative AI.'

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
# def print_prompt(prompt_text):
#   print("Prompt - ", prompt_text)
#   return prompt_text

In [ ]:
# rag_chain_with_print = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
#              | prompt
#              | RunnableLambda(print_prompt)
#              | llm
#              | StrOutputParser())

In [ ]:
rag_chain_with_print.invoke("What all projects are covered in the course?")

Prompt -  messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What all projects are covered in the course? \nContext: course be taught in?Do you have a mobile application?Can I watch the class recordings or go through notes offline?Do you provide certificate for the course?Can I get invoices for reimbursement?I have more questions, how can I reach out to you?© 2025 Copyright Educosys. All rights reserved.\ninterested in AI and ML!Read moreManika KaushikSenior Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.Frequently asked questionsIs this a Live or Recorded Course?When will the next Live batch be launched?What if I am interested in learning Live onl

'The projects covered in the course include implementing a GAN to generate handwritten digits, training a VAE to generate faces using the CelebA dataset, code Transformer from scratch, and sentiment analysis using BERT. There are also mini-projects related to TensorFlow, TensorBoard, and various deep generative models. The course covers a range of topics related to generative AI and provides hands-on learning opportunities.'

In [ ]:
## Testing of openai api key

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

In [2]:
from openai import OpenAI

# Create client (API key is read from environment variable)
client = OpenAI()

print("🤖 Simple Chatbot — type 'exit' to quit\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye! 👋")
        break

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a friendly chatbot that helps users."},
            {"role": "user", "content": user_input}
        ]
    )

    reply = response.choices[0].message.content
    print(f"Chatbot: {reply}\n")


🤖 Simple Chatbot — type 'exit' to quit

You: hi
Chatbot: Hello! How can I assist you today?

You: what are some of the latest news
Chatbot: I'm unable to provide real-time news updates, but I can summarize major events and trends up to October 2023. If you're looking for the latest information, I recommend checking reputable news websites or apps. Is there a specific topic or area you're interested in? I can help summarize recent trends or events in that area!

You: which are some of the free tier gpt models?
Chatbot: As of October 2023, some platforms offer free tier access to various GPT models. Here are a few options where you might find free access to GPT-like models:

1. **OpenAI's ChatGPT**: OpenAI offers a free tier for users to access its models on the ChatGPT platform. While there are paid options (such as ChatGPT Plus), users can still use a basic version for free, typically with some usage limits.

2. **Hugging Face**: Hugging Face provides a wide range of language models, i

KeyboardInterrupt: Interrupted by user